In [1]:
import csv
import tensorflow as tf
import os
import numpy as np
import scipy.io as sio
import sys
import gc
import pandas as pd
from sklearn import linear_model
from nn.td_prediction_lstm_V3 import td_prediction_lstm_V3
from nn.td_prediction_lstm_V4 import lstm_V4
from utils import handle_trace_length, get_together_training_batch, compromise_state_trace_length
from configuration import MODEL_TYPE, MAX_TRACE_LENGTH, FEATURE_NUMBER, BATCH_SIZE, GAMMA, H_SIZE, \
    model_train_continue, FEATURE_TYPE, ITERATE_NUM, learning_rate, SPORT

2023-12-26 11:18:48.920405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from nn import td_prediction_lstm_V3
from nn import td_prediction_lstm_V4
from utils import handle_trace_length, get_together_training_batch, compromise_state_trace_length
from configuration import MODEL_TYPE, MAX_TRACE_LENGTH, FEATURE_NUMBER, BATCH_SIZE, GAMMA, H_SIZE, \
    model_train_continue, FEATURE_TYPE, ITERATE_NUM, learning_rate, SPORT

In [3]:
save_mother_dir = "save-no-goal"
reward_name = ""
state_input_name = ""
state_trace_length_name = ""

In [4]:
LOG_DIR = save_mother_dir + "/models/hybrid_sl_log_NN/Scale-three-cut_together_log_train_feature" + str(
    FEATURE_TYPE) + "_batch" + str(
    BATCH_SIZE) + "_iterate" + str(
    ITERATE_NUM) + "_lr" + str(
    learning_rate) + "_" + str(MODEL_TYPE) + "_MaxTL" + str(MAX_TRACE_LENGTH)
SAVED_NETWORK = save_mother_dir + "/models/hybrid_sl_saved_NN/Scale-three-cut_together_saved_networks_feature" + str(
    FEATURE_TYPE) + "_batch" + str(
    BATCH_SIZE) + "_iterate" + str(
    ITERATE_NUM) + "_lr" + str(
    learning_rate) + "_" + str(MODEL_TYPE) + "_MaxTL" + str(MAX_TRACE_LENGTH)
DATA_STORE = "data-dir-no-goal"

DIR_GAMES_ALL = os.listdir(DATA_STORE)
#DIR_GAMES_ALL = DIR_GAMES_ALL[1:]
DIR_GAMES_ALL = [f for f in DIR_GAMES_ALL if "dir-game" in f]
number_of_total_game = len(DIR_GAMES_ALL)

print(DIR_GAMES_ALL)

['dir-game5354538', 'dir-game5354533', 'dir-game5354712', 'dir-game5354713', 'dir-game5354714']


In [5]:
def write_game_average_csv(data_record):
    """
    write the cost of training
    :param data_record: the recorded cost dict
    """
    try:
        if os.path.exists(LOG_DIR + '/avg_cost_record.csv'):
            with open(LOG_DIR + '/avg_cost_record.csv', 'a') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for record in data_record:
                    writer.writerow(record)
        else:
            with open(LOG_DIR + '/avg_cost_record.csv', 'w') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for record in data_record:
                    writer.writerow(record)
    except:
        if os.path.exists(LOG_DIR + '/avg_cost_record2.csv'):
            with open(LOG_DIR + '/avg_cost_record.csv', 'a') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for record in data_record:
                    writer.writerow(record)
        else:
            with open(LOG_DIR + '/avg_cost_record2.csv', 'w') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for record in data_record:
                    writer.writerow(record)

In [6]:
def train_network(sess, model):
    """
    training thr neural network game by game
    :param sess: session of tf
    :param model: nn model
    :return:
    """
    
    game_number = 0
    global_counter = 0
    converge_flag = False

    # loading network
    game_number = 0
    saver = tf.train.Saver()
    merge = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    sess.run(tf.global_variables_initializer())
    if model_train_continue:
        checkpoint = tf.train.get_checkpoint_state(SAVED_NETWORK)
        if checkpoint and checkpoint.model_checkpoint_path:
            check_point_game_number = int((checkpoint.model_checkpoint_path.split("-"))[-1])
            game_number_checkpoint = check_point_game_number % number_of_total_game
            game_number = check_point_game_number
            game_starting_point = 0
            saver.restore(sess, checkpoint.model_checkpoint_path)
            print("Successfully loaded:", checkpoint.model_checkpoint_path)
        else:
            print("Could not find old network weights")

    game_diff_record_all = []

    while True:
        game_diff_record_dict = {}
        iteration_now = game_number / number_of_total_game + 1
        game_diff_record_dict.update({"Iteration": iteration_now})
        if converge_flag:
            break
        elif game_number >= number_of_total_game * ITERATE_NUM:
            break
        else:
            converge_flag = True
        for dir_game in DIR_GAMES_ALL:#試合毎のループ
            if checkpoint and checkpoint.model_checkpoint_path:
                if model_train_continue:  # go the check point data
                    game_starting_point += 1
                    if game_number_checkpoint + 1 > game_starting_point:
                        continue

            v_diff_record = []
            game_number += 1
            game_cost_record = []
            game_files = os.listdir(DATA_STORE + "/" + dir_game)
            
            reward_name = None
            state_input_name = None
            state_trace_length_name = None
            for filename in game_files:
                if "dynamic_rnn_reward" in filename:
                    reward_name = filename
                elif "dynamic_rnn_input" in filename:
                    state_input_name = filename
                elif "trace" in filename:
                    state_trace_length_name = filename
                    
            if reward_name is None:
                print(f"Cannot find a file with 'dynamic_rnn_reward' in directory {DATA_STORE}/{dir_game}. Skipping this directory.")
                continue
            if state_input_name is None:
                print(f"Cannot find a file with 'dynamic_rnn_input' in directory {DATA_STORE}/{dir_game}. Skipping this directory.")
                continue
            if state_trace_length_name is None:
                print(f"Cannot find a file with 'trace' in directory {DATA_STORE}/{dir_game}. Skipping this directory.")
                continue

            state_input = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + state_input_name)
            reward = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + reward_name)
            state_trace_length = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + state_trace_length_name)
            state_trace_length = (state_trace_length['hybrid_trace_length'])[0]
            state_input = (state_input['dynamic_feature_input'])
            try:
                reward = reward['dynamic_rnn_reward']
            except:
                print("\n" + dir_game)
                raise ValueError("reward wrong")
            #print(state_trace_length)
            state_trace_length = handle_trace_length(state_trace_length)
            state_trace_length, state_input, reward = compromise_state_trace_length(state_trace_length, state_input,
                                                                                    reward, MAX_TRACE_LENGTH)
            
            #print("\n load file" + str(dir_game) + " success")
            reward_count = sum(reward)
            #print("reward number" + str(reward_count))
            if len(state_input) != len(reward) or len(state_trace_length) != len(reward):
                raise Exception('state length does not equal to reward length')

            train_len = len(state_input)
            train_number = 0
            s_t0 = state_input[train_number]
            train_number += 1
            
            while True:#ミニバッチ毎にトレーニング
                # try:
                batch_return, train_number, s_tl, current_batch_length = get_together_training_batch(s_t0,
                                                                               state_input,
                                                                               reward,
                                                                               train_number,
                                                                               train_len,
                                                                               state_trace_length,
                                                                               BATCH_SIZE)
                
                s_t0_batch = [d[0] for d in batch_return]
                s_t1_batch = [d[1] for d in batch_return]
                r_t_batch = [d[2] for d in batch_return]
                trace_t0_batch = [d[3] for d in batch_return]
                trace_t1_batch = [d[4] for d in batch_return]
                y_batch = []
                
                #print('s_t0_batch : ', s_t0_batch)
                #print('s_t1_batch : ', s_t1_batch)
                #print('trace_t0_batch : ', trace_t0_batch)
                #print('trace_t1_batch : ', trace_t1_batch)
                #print('current_batch_length : ', current_batch_length)
                

                [outputs_t1, readout_t1_batch] = sess.run([model.outputs, model.read_out],
                                                          feed_dict={model.trace_lengths: trace_t1_batch,
                                                                     model.rnn_input: s_t1_batch})

                for i in range(0, len(batch_return)):
                    terminal = batch_return[i][5]
                    cut = batch_return[i][6]
                    # if terminal, only equals reward
                    if terminal or cut:
                        y_home = float((r_t_batch[i])[0])
                        y_away = float((r_t_batch[i])[1])
                        y_end = float((r_t_batch[i])[2])
                        y_batch.append([y_home, y_away, y_end])
                        break
                    else:
                        y_home = float((r_t_batch[i])[0]) + GAMMA * ((readout_t1_batch[i]).tolist())[0]
                        y_away = float((r_t_batch[i])[1]) + GAMMA * ((readout_t1_batch[i]).tolist())[1]
                        y_end = float((r_t_batch[i])[2]) + GAMMA * ((readout_t1_batch[i]).tolist())[2]
                        y_batch.append([y_home, y_away, y_end])
                
                y_batch = np.asarray(y_batch)
                [diff, read_out, cost_out, summary_train, _] = sess.run(
                    [model.diff, model.read_out, model.cost, merge, model.train_step],
                    feed_dict={model.y: y_batch,
                               model.trace_lengths: trace_t0_batch,
                               model.rnn_input: s_t0_batch})
                

                v_diff_record.append(diff)

                if cost_out > 0.0001:
                    converge_flag = False
                global_counter += 1
                game_cost_record.append(cost_out)
                train_writer.add_summary(summary_train, global_step=global_counter)
                s_t0 = s_tl

                # print info
                if terminal or ((train_number - 1) / BATCH_SIZE) % 5 == 1:
                    print("TIMESTEP:", train_number, "Game:", game_number)
                    home_avg = sum(read_out[:, 0]) / len(read_out[:, 0])
                    away_avg = sum(read_out[:, 1]) / len(read_out[:, 1])
                    end_avg = sum(read_out[:, 2]) / len(read_out[:, 2])
                    print("home average:{0}, away average:{1}, end average:{2}".format(str(home_avg), str(away_avg),
                                                                                       str(end_avg)))
                    print("cost of the network is" + str(cost_out))
                    print()

                if terminal:
                    # save progress after a game
                    saver.save(sess, SAVED_NETWORK + '/' + SPORT + '-game-', global_step=game_number)
                    v_diff_record_average = sum(v_diff_record) / len(v_diff_record)
                    game_diff_record_dict.update({dir_game: v_diff_record_average})
                    break

                    # break
            cost_per_game_average = sum(game_cost_record) / len(game_cost_record)
            write_game_average_csv([{"iteration": str(game_number / number_of_total_game + 1), "game": game_number,
                                     "cost_per_game_average": cost_per_game_average}])

    game_diff_record_all.append(game_diff_record_dict)

In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def train_start():
    if not os.path.isdir(LOG_DIR):
        #print(LOG_DIR)
        os.makedirs(LOG_DIR)
    if not os.path.isdir(SAVED_NETWORK):
        #print(SAVED_NETWORK)
        os.makedirs(SAVED_NETWORK)

    sess = tf.InteractiveSession()
    if MODEL_TYPE == "v3":
        nn = td_prediction_lstm_V3(FEATURE_NUMBER, H_SIZE, MAX_TRACE_LENGTH, learning_rate)
    elif MODEL_TYPE == "v4":
        nn = lstm_V4(FEATURE_NUMBER, H_SIZE, MAX_TRACE_LENGTH, learning_rate)
    else:
        raise ValueError("MODEL_TYPE error")
    train_network(sess, nn)
    
if __name__ == '__main__':
    train_start()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2023-12-26 11:19:00.607178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/rikito-e/Documents/SportMeme/sports-data-analysis-main/calc_metrics/src/2023reinforcement-hockey/nn/td_prediction_lstm_V4.py:17: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  self.lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=H_SIZE * 2, state_is_tuple=True, initializer=tf.random_uniform_initializer(-0.05, 0.05))
2023-12-26 11:19:01.008140: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Could not find old network weights
TIMESTEP: 33 Game: 1
home average:-0.009607785465050256, away average:-0.027363497545593418, end average:-0.018486539309378713
cost of the network is0.00027234133

TIMESTEP: 193 Game: 1
home average:0.015692887232944486, away average:-0.007362778289461858, end average:0.0025002462607517373
cost of the network is0.0001031975

TIMESTEP: 353 Game: 1
home average:0.015976130226590612, away average:-0.018889730110004166, end average:0.012508189615346055
cost of the network is0.0002532689

TIMESTEP: 513 Game: 1
home average:-0.03042079696751898, away average:-0.014059464570891578, end average:-0.01689382111726445
cost of the network is0.00018650586

TIMESTEP: 673 Game: 1
home average:-0.009389250091771828, away average:-0.007900269569745433, end average:-0.002980131332151359
cost of the network is6.781562e-05

TIMESTEP: 833 Game: 1
home average:-0.008032685047510313, away average:-0.015470604717847891, end average:-0.012472051497752545
cost of the network i

TIMESTEP: 33 Game: 7
home average:0.06448334082961082, away average:0.047399042174220085, end average:0.035048361256485805
cost of the network is0.00026570354

TIMESTEP: 193 Game: 7
home average:0.06612590723671019, away average:0.04914915364497574, end average:0.03306842426536605
cost of the network is0.00022186308

TIMESTEP: 353 Game: 7
home average:0.12601666885893792, away average:0.059967698296532035, end average:0.05105171998729929
cost of the network is0.00029802692

TIMESTEP: 513 Game: 7
home average:0.08796534535940737, away average:0.07399797718971968, end average:0.046259925002232194
cost of the network is0.00031952103

TIMESTEP: 673 Game: 7
home average:0.07030078186653554, away average:0.060244900989346206, end average:0.04597265552729368
cost of the network is0.00012575762

TIMESTEP: 1717 Game: 7
home average:0.19381424571786607, away average:0.12026013513760907, end average:0.15591283408658846
cost of the network is0.019325104

TIMESTEP: 33 Game: 8
home average:0.0944334

TIMESTEP: 353 Game: 14
home average:0.0958027207525447, away average:0.16552245570346713, end average:0.08756863744929433
cost of the network is0.00031761292

TIMESTEP: 513 Game: 14
home average:0.11105716554448009, away average:0.14767134794965386, end average:0.09938633139245212
cost of the network is0.00012448944

TIMESTEP: 673 Game: 14
home average:0.10507436073385179, away average:0.15353632904589176, end average:0.1089146516751498
cost of the network is5.381936e-05

TIMESTEP: 833 Game: 14
home average:0.10487387469038367, away average:0.16401177691295743, end average:0.11866996483877301
cost of the network is0.000160637

TIMESTEP: 993 Game: 14
home average:0.11632902175188065, away average:0.15873625222593546, end average:0.13180572306737304
cost of the network is0.0002349931

TIMESTEP: 1153 Game: 14
home average:0.12601775699295104, away average:0.15900446102023125, end average:0.12617723806761205
cost of the network is0.00018948225

TIMESTEP: 1313 Game: 14
home average:0.123133

...

TIMESTEP: 993 Game: 124
home average:0.21556796273216605, away average:0.43678381387144327, end average:0.24330936139449477
cost of the network is0.00024673215

TIMESTEP: 1153 Game: 124
home average:0.23756881477311254, away average:0.4106238642707467, end average:0.2651804806664586
cost of the network is0.00025454702

TIMESTEP: 1313 Game: 124
home average:0.23951481329277158, away average:0.41360845882445574, end average:0.27368326066061854
cost of the network is0.00033475252

TIMESTEP: 1473 Game: 124
home average:0.2520449352450669, away average:0.3927721530199051, end average:0.28951906273141503
cost of the network is0.0010179476

TIMESTEP: 1633 Game: 124
home average:0.22308514406904578, away average:0.3988412991166115, end average:0.32828086521476507
cost of the network is0.00047568697

TIMESTEP: 1776 Game: 124
home average:0.248173700645566, away average:0.3348434614017606, end average:0.3796836994588375
cost of the network is0.0056731533

TIMESTEP: 33 Game: 125
home average:0.19

TIMESTEP: 993 Game: 130
home average:0.22453294601291418, away average:0.4224307518452406, end average:0.25902868853881955
cost of the network is0.00024697496

TIMESTEP: 1664 Game: 130
home average:0.18078368976712228, away average:0.38307322561740875, end average:0.4991060942411423
cost of the network is0.0069194986

TIMESTEP: 33 Game: 131
home average:0.18634406011551619, away average:0.5026573715731502, end average:0.17542480584234
cost of the network is0.0006838792

TIMESTEP: 193 Game: 131
home average:0.2156392582692206, away average:0.4911417104303837, end average:0.177934093400836
cost of the network is0.00024877983

TIMESTEP: 353 Game: 131
home average:0.1830651443451643, away average:0.5069170258939266, end average:0.21228289371356368
cost of the network is0.00038731168

TIMESTEP: 513 Game: 131
home average:0.19155009603127837, away average:0.500653900206089, end average:0.21201390540227294
cost of the network is0.00014866613

TIMESTEP: 673 Game: 131
home average:0.20608506770

TIMESTEP: 33 Game: 137
home average:0.19185587763786316, away average:0.4703050274401903, end average:0.18613639706745744
cost of the network is0.00037467552

TIMESTEP: 193 Game: 137
home average:0.20686864061281085, away average:0.4610458239912987, end average:0.18468111054971814
cost of the network is0.0003232411

TIMESTEP: 353 Game: 137
home average:0.22185323620215058, away average:0.4596298411488533, end average:0.21847765147686005
cost of the network is0.00017674208

TIMESTEP: 513 Game: 137
home average:0.22136070346459746, away average:0.4700428396463394, end average:0.21090819593518972
cost of the network is0.00034770058

TIMESTEP: 673 Game: 137
home average:0.2089478811249137, away average:0.4517184291034937, end average:0.2334116385318339
cost of the network is0.00020897873

TIMESTEP: 1717 Game: 137
home average:0.4513311577694757, away average:-0.027405430163655962, end average:0.6453656000750405
cost of the network is0.004280141

TIMESTEP: 33 Game: 138
home average:0.223131

TIMESTEP: 353 Game: 144
home average:0.21845413418486714, away average:0.47952612675726414, end average:0.18202010309323668
cost of the network is0.00020050176

TIMESTEP: 513 Game: 144
home average:0.22860303660854697, away average:0.46931225433945656, end average:0.19655346916988492
cost of the network is0.00026324065

TIMESTEP: 673 Game: 144
home average:0.22291513672098517, away average:0.4730670964345336, end average:0.21330105513334274
cost of the network is0.00018084445

TIMESTEP: 833 Game: 144
home average:0.21461620414629579, away average:0.4709868337959051, end average:0.22229831665754318
cost of the network is0.00022522143

TIMESTEP: 993 Game: 144
home average:0.21921611158177257, away average:0.46085924841463566, end average:0.24040426965802908
cost of the network is0.00022075277

TIMESTEP: 1153 Game: 144
home average:0.23891051346436143, away average:0.41802272107452154, end average:0.2594559430144727
cost of the network is0.00036764835

TIMESTEP: 1313 Game: 144
home averag

TIMESTEP: 353 Game: 150
home average:0.22475446201860905, away average:0.48317194916307926, end average:0.19571259105578065
cost of the network is0.00020644754

TIMESTEP: 513 Game: 150
home average:0.22480914182960987, away average:0.4756325650960207, end average:0.20894577773287892
cost of the network is0.0004110059

TIMESTEP: 673 Game: 150
home average:0.23743860190734267, away average:0.4564379006624222, end average:0.22593655483797193
cost of the network is0.00019686756

TIMESTEP: 833 Game: 150
home average:0.22806528210639954, away average:0.45147031638771296, end average:0.25081341434270144
cost of the network is0.000322488

TIMESTEP: 993 Game: 150
home average:0.23957023629918694, away average:0.43538029305636883, end average:0.262713351752609
cost of the network is0.00017612272

TIMESTEP: 1664 Game: 150
home average:0.17892245315015315, away average:0.3614388987421989, end average:0.5050166308879852
cost of the network is0.006240735



In [2]:
import random
import pandas as pd

a_list = [random.choice([0, 1]) for _ in range(60)]
b_list = [random.choice([0, 1]) for _ in range(60)]
c_list = [random.choice([0, 1]) for _ in range(60)]

a = a_list
b = b_list
c = c_list

events = {'match_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        'side': [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1],
        'goal': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         'a':a,
         'b':b,
         'c':c}
events = pd.DataFrame(events)
events

,match_id,side,goal,a,b,c
0,0,0,0,1,1,0
1,0,0,0,1,1,1
2,0,0,0,0,1,1
3,0,1,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,1,0
6,0,0,0,1,0,1
7,0,1,0,1,0,1
8,0,1,0,0,1,0
9,0,1,1,1,1,0


In [3]:
#プレーに分解
events['team_changed'] = (events['side'] != events['side'].shift()).astype(int)
events['team_changed'] = events['team_changed'].cumsum() * events['team_changed']
events['team_changed'] = events['team_changed'].where(events['team_changed'] != 0, None)

events['team_changed'] = events['team_changed'].ffill()

#hybrid_trace_lengthの作成
hybrid_trace_length = events['team_changed'].value_counts().sort_index().values.tolist()

trace_length = {
    "hybrid_trace_length": hybrid_trace_length
}
trace_length

{'hybrid_trace_length': [3,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  13,
  1,
  3,
  3]}

In [4]:
desired_order = ["match_id", "goal", "side", "team_changed"]
remaining_columns = [col for col in events.columns if col not in desired_order]

events = events[desired_order + remaining_columns]

In [5]:
events_index = events.loc[:, 'match_id':'goal']
events_input_raw = events.drop(columns=events_index.columns)
events_index['team_changed'] = events['team_changed']
events_index['side'] = events['side']

In [6]:
#state_inputの作成
events_input_columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

placeholder_list = [0] * (events_input_raw.shape[1] - 1)

all_data = []

for i, row in events_input_raw.iterrows():
    subset = events_input_raw.iloc[:i+1]
    filtered_rows = subset[subset['team_changed'] == row['team_changed']]
    
    rows_as_lists = filtered_rows.iloc[:, [0] + list(range(2, filtered_rows.shape[1]))].values.tolist()
    rows_as_lists = rows_as_lists[-10:]
    
    while len(rows_as_lists) < 10:
        rows_as_lists.append(placeholder_list)
    
    all_data.append(rows_as_lists)

events_input = pd.DataFrame(all_data, columns=events_input_columns)

state_input = {
    "dynamic_feature_input": events_input.values.tolist()
}

state_input

{'dynamic_feature_input': [[[0, 1, 1, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[0, 1, 1, 0],
   [0, 1, 1, 1],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[0, 1, 1, 0],
   [0, 1, 1, 1],
   [0, 0, 1, 1],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[1, 0, 1, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[0, 0, 0, 1],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[0, 0, 0, 1],
   [0, 0, 1, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
 

In [7]:
#state_inputの分解
state_split = {}
unique_match_ids = events_index['match_id'].unique()

for match_id in unique_match_ids:
    idx = events_index[events_index['match_id'] == match_id].index.tolist()
    state_split[f"state_input{match_id}"] = {
        "dynamic_feature_input": [state_input["dynamic_feature_input"][i] for i in idx]
    }

i = 0
while i < len(unique_match_ids):
    vars()[f"state_input{unique_match_ids[i]}"] = state_split[f"state_input{unique_match_ids[i]}"]
    i += 1

len(state_split['state_input1']['dynamic_feature_input'])

30

In [8]:
#特徴量の数
print(len(events_input.iloc[0, 0]))

4


In [9]:
#rewardの作成
events_index['iteration'] = events_index.groupby((events_index['team_changed'] != events_index['team_changed'].shift()).cumsum()).cumcount() + 1
reward = [[0]*10 for _ in range(len(events_index))]

for index, row in events_index[(events_index['goal'] == 1) & (events_index['side'] == 1)].iterrows():
    if row['iteration'] <= 10:
        goal_iteration = int(row['iteration'])
        reward[index][goal_iteration-1] = 1
    else:
        reward[index][9] = 1

for index, row in events_index[(events_index['goal'] == 1) & (events_index['side'] == 0)].iterrows():
    if row['iteration'] <= 10:
        goal_iteration = int(row['iteration'])
        reward[index][goal_iteration-1] = -1
    else:
        reward[index][9] = -1

reward = {
    "dynamic_rnn_reward": reward
}

reward

{'dynamic_rnn_reward': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
#rewardの分解
rewards_split = {}
unique_match_ids = events_index['match_id'].unique()

for match_id in unique_match_ids:
    idx = events_index[events_index['match_id'] == match_id].index.tolist()
    rewards_split[f"reward{match_id}"] = {
        "dynamic_rnn_reward": [reward["dynamic_rnn_reward"][i] for i in idx]
    }
    
i = 0
while i < len(unique_match_ids):
    vars()[f"reward{unique_match_ids[i]}"] = rewards_split[f"reward{unique_match_ids[i]}"]
    i += 1

print(rewards_split['reward1'])

{'dynamic_rnn_reward': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [11]:
#hybrid_trace_lengthの分解
def get_actual_consecutive_counts_updated(group):
    vals = group.tolist()
    counts = []
    current_count = 1

    for i in range(1, len(vals)):
        if vals[i] == vals[i-1]:
            current_count += 1
        else:
            counts.append(current_count)
            current_count = 1
    counts.append(current_count)
    return counts

actual_consecutive_counts_updated = events_index.groupby('match_id')['team_changed'].apply(get_actual_consecutive_counts_updated)

trace_length_split = {}
unique_match_ids = events_index['match_id'].unique()

for match_id in unique_match_ids:
    trace_length_split[f"hybrid_trace_length{match_id}"] = {
        "hybrid_trace_length": actual_consecutive_counts_updated[match_id]
    }

trace_length_split["hybrid_trace_length1"]

{'hybrid_trace_length': [3, 1, 3, 3, 13, 1, 3, 3]}

In [12]:
#入力データの保存
from decimal import Decimal

def convert_decimal_to_float(input_data):
    """Convert decimal.Decimal to float recursively in a dictionary."""
    if isinstance(input_data, dict):
        return {key: convert_decimal_to_float(value) for key, value in input_data.items()}
    elif isinstance(input_data, list):
        return [convert_decimal_to_float(item) for item in input_data]
    elif isinstance(input_data, tuple):
        return tuple(convert_decimal_to_float(item) for item in input_data)
    elif isinstance(input_data, (float, int)):
        return input_data
    elif isinstance(input_data, Decimal):
        return float(input_data)
    else:
        return input_data
    
state_input = convert_decimal_to_float(state_input)
        
# 入力データの保存
for match_id in unique_match_ids:
    directory_path = f"data-dir-check/dir-game{match_id}"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    sio.savemat(os.path.join(directory_path, f"hybrid_trace_length_{match_id}.mat"), trace_length_split[f"hybrid_trace_length{match_id}"])
    sio.savemat(os.path.join(directory_path, f"dynamic_rnn_input_{match_id}.mat"), state_split[f"state_input{match_id}"])
    sio.savemat(os.path.join(directory_path, f"dynamic_rnn_reward_{match_id}.mat"), rewards_split[f"reward{match_id}"])